In [ ]:
# Imports

from IPython.display import HTML

import matplotlib.pyplot as plt
import seaborn as sns

from environ import get_params
from db import query_single_row, query_data_frame
from queries import current_study_year

In [ ]:
params = get_params(
    STUDENT_ID = 1000,
    CURRENT_STUDY_YEAR = current_study_year()
)
params.STUDENT_ID = int(params.STUDENT_ID)
params.CURRENT_STUDY_YEAR = int(params.CURRENT_STUDY_YEAR)

In [ ]:
student = query_single_row("""
    SELECT
        uniid,
        first_name,
        last_name,
        date_part('year', CURRENT_DATE) - year_of_birth AS age,
        curriculum,
        study_form,
        is_compulsory,
        grade
    FROM study INNER JOIN student USING (student_id)
    WHERE study_year = %s AND student_id = %s
""", (params.CURRENT_STUDY_YEAR, params.STUDENT_ID))

In [ ]:
HTML(f"""
    <h1 style="font-size: 3em">{student.first_name} {student.last_name} (<i>{student.uniid}<i/>)<h1/>
    <h2 style="font-size: 0.6em" >General data</h2>
    <table>
        <tr><td><strong>Curriculum:<strong/><td/>{student.curriculum} ({"compulsory" if student.is_compulsory else "not compulsory"})<td><td/><tr/>
        <tr><td><strong>Study form:<strong/><td/>{student.study_form}<td><td/><tr/>
        <tr><td><strong>Age:<strong/><td/>{student.age}<td><td/><tr/>
    <table/>
""")

## Performance on course

In [ ]:
score_on_weeks = query_data_frame("""
    SELECT
        week_number,
        score_on_week(%(student_id)s, %(study_year)s, week_number) AS student_score,
        (SELECT avg_score FROM weekly_stats WHERE weekly_stats.study_year = %(study_year)s AND weekly_stats.week = week_number) AS course_avg_score
    FROM generate_series(1, 16) as week (week_number)
""", {"student_id": params.STUDENT_ID, "study_year": params.CURRENT_STUDY_YEAR})
score_on_weeks

plt.figure(figsize=(13,6))
plt.plot(score_on_weeks["week_number"], score_on_weeks["student_score"], label="student's score")
plt.plot(score_on_weeks["week_number"], score_on_weeks["course_avg_score"], label="course mean score")
plt.legend()
plt.grid()
plt.xlabel("study week")
plt.ylabel("score");
